In [1]:
!unzip "/content/drive/MyDrive/Datasets/Object Detection/OD_xml_tiny.zip" -d "/content/sample_data/OD_xml_tiny"

Archive:  /content/drive/MyDrive/Datasets/Object Detection/OD_xml_tiny.zip
replace /content/sample_data/OD_xml_tiny/validation/2007_000584.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
%cd "/content/drive/MyDrive/Object Detection/Yolo/Yolo - Tensorflow 2 - v2"

/content/drive/MyDrive/Object Detection/Yolo/Yolo - Tensorflow 2 - v2


In [3]:
import os
import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from models.yolov3 import YOLOv3Encoder, YOLOv3Decoder
from models.yolo import YOLO
from losses.yolo_loss import YOLOv3Loss
from data_utils.data_flow import YoloDatasets
from callbacks.warmup_lr import AdvanceWarmUpLearningRate
from callbacks.evaluate_map import mAPEvaluate
from utils.files import verify_folder
from utils.logger import logger
from visualizer.visual_image import visual_image
from visualizer.visual_value import tensor_value_info

In [4]:
# tf.config.run_functions_eagerly(True)

In [5]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path, encoding='utf-8') as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    anchors = np.array(anchors).reshape(-1, 2)
    return anchors, len(anchors)

In [6]:
def get_classes(classes_path):
    with open(classes_path, encoding='utf-8') as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names, len(class_names)

In [7]:
classes_path                        = '/content/sample_data/OD_xml_tiny/object_names.txt'

train_annotation_path               = '/content/sample_data/OD_xml_tiny/train.txt'
val_annotation_path                 = '/content/sample_data/OD_xml_tiny/validation.txt'
anchors_path                        = './configs/yolo_anchors.txt'
anchors_mask                        = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]

strides = [8, 16, 32]


input_shape                         = [416, 416, 3]

Init_Epoch                          = 0
Freeze_Epoch                        = 50
Freeze_batch_size                   = 16

UnFreeze_Epoch                      = 300
Unfreeze_batch_size                 = 8

Freeze_Train                        = False

Init_lr                             = 1e-2
Min_lr                              = Init_lr * 0.01

optimizer_type                      = "sgd"
momentum                            = 0.937
weight_decay                        = 5e-4

lr_decay_type                       = 'cos'

save_period                         = 10

saved_path                            = '/content/sample_data/saved_weights/'

load_type                          = None

weight_objects                    = [        
                                    {
                                        'path': './saved_weights/checkpoints/last_epoch_weights',
                                        'stage': 'full',
                                        'custom_objects': None
                                    }
                                ]

eval_flag                           = True
eval_period                         = 10

num_workers                         = 1

batch_size = 8
learning_rate = 1e-2
lr_init = 1e-2
lr_end = lr_init * 0.01

epochs = UnFreeze_Epoch

class_names, num_classes            = get_classes(classes_path)
anchors, num_anchors                = get_anchors(anchors_path)

In [8]:
def create_folder_weights(saved_dir):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    TRAINING_TIME_PATH = saved_dir + current_time
    access_rights = 0o755
    try:
        os.makedirs(TRAINING_TIME_PATH, access_rights)
        logger.info("Successfully created the directory %s" % TRAINING_TIME_PATH)
        return verify_folder(TRAINING_TIME_PATH)
    except: 
        logger.error("Creation of the directory %s failed" % TRAINING_TIME_PATH)

In [9]:
def train(input_shape, anchors, batch_size, num_classes, anchors_mask, 
          strides, learning_rate, lr_init, lr_end, epochs, load_type, weight_objects, saved_path):
    TRAINING_TIME_PATH = create_folder_weights(saved_path)

    with open(train_annotation_path, encoding='utf-8') as f:
        train_lines = f.readlines()
    with open(val_annotation_path, encoding='utf-8') as f:
        val_lines   = f.readlines()
    num_train   = len(train_lines)
    num_val     = len(val_lines)

    train_dataloader    = YoloDatasets(train_lines, input_shape, anchors, anchors_mask, strides, batch_size, num_classes, train=True)
    val_dataloader      = YoloDatasets(val_lines, input_shape, anchors, anchors_mask, strides, batch_size, num_classes, train=False)

    encoder = YOLOv3Encoder(num_classes, num_anchor=3, darknet_weight=None)
    decoder = YOLOv3Decoder(anchors,
                            num_classes,
                            input_shape,
                            anchors_mask,
                            max_boxes=100,
                            confidence=0.05,
                            nms_iou=0.5,
                            letterbox_image=True)

    model = YOLO(encoder, decoder)

    if load_type and weight_objects:
        if load_type == "weights":
            model.load_weights(weight_objects)
        elif load_type == "models":
            model.load_models(weight_objects)


    loss = YOLOv3Loss(input_shape, anchors, anchors_mask, num_classes, 
                      balance     = [0.4, 1.0, 4],
                      box_ratio   = 0.05, 
                      obj_ratio   = 5 * (input_shape[0] * input_shape[1]) / (416 ** 2),
                      cls_ratio   = 1 * (num_classes / 80))
    
    nbs             = 64
    lr_limit_max    = 5e-2 
    lr_limit_min    = 5e-4
    Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
    Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)

    warmup_lr = AdvanceWarmUpLearningRate(lr_init=Init_lr_fit, lr_end=Min_lr_fit, epochs=epochs)
    eval_callback = mAPEvaluate(input_shape, class_names, val_lines, TRAINING_TIME_PATH, eval_flag=True, period=10)
    callbacks = [warmup_lr, eval_callback]

    optimizer = SGD(learning_rate=Init_lr_fit, momentum=momentum, nesterov=True)

    model.compile(optimizer=optimizer, loss=loss)
    

    epoch_step      = num_train // batch_size
    epoch_step_val  = num_val // batch_size

    model.fit(train_dataloader,
              steps_per_epoch     = epoch_step,
              validation_data     = val_dataloader,
              validation_steps    = epoch_step_val,
              epochs              = epochs,
              callbacks           = callbacks)
    model.save_weights(TRAINING_TIME_PATH + 'best_weights', save_format="tf")

In [10]:
train(input_shape, anchors, batch_size, num_classes, 
      anchors_mask, strides, learning_rate, lr_init, lr_end, 
      epochs, load_type, weight_objects, saved_path)

2022-09-10 19:28:08,104 - INFO - Successfully created the directory /content/sample_data/saved_weights/20220910-192808
INFO:YOLO:Successfully created the directory /content/sample_data/saved_weights/20220910-192808


Epoch 1/300
8/8 [==============================] - 23s 1s/step - total_loss: 20.5719 - learning_rate: 1.2500e-04 - val_total_loss: 22.2682
Epoch 2/300
8/8 [==============================] - 5s 666ms/step - total_loss: 14.5512 - learning_rate: 2.5000e-04 - val_total_loss: 21.6360
Epoch 3/300
8/8 [==============================] - 5s 679ms/step - total_loss: 8.1625 - learning_rate: 6.2500e-04 - val_total_loss: 20.2980
Epoch 4/300
8/8 [==============================] - 5s 678ms/step - total_loss: 5.0418 - learning_rate: 0.0012 - val_total_loss: 18.7803
Epoch 5/300
8/8 [==============================] - 5s 680ms/step - total_loss: 4.0483 - learning_rate: 0.0012 - val_total_loss: 16.9710
Epoch 6/300
8/8 [==============================] - 5s 665ms/step - total_loss: 3.8328 - learning_rate: 0.0012 - val_total_loss: 15.5087
Epoch 7/300
2/8 [======>.......................] - ETA: 3s - total_loss: 3.7556 - learning_rate: 0.0012

KeyboardInterrupt: ignored